In [1]:
#importing the libraries
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso
import statsmodels.api as sm
import pyreadr
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import explained_variance_score
from sklearn import metrics

In [17]:
result = pyreadr.read_r('Movies.RData')# also works for Rds
print(result.keys())
df = pd.DataFrame(result['movies'], columns=result['movies'].keys() )
df.shape

odict_keys(['movies'])


(651, 32)

In [18]:
df.shape[0]

651

In [19]:
df.set_index("title", inplace=True)

In [20]:
df_1 = df.loc[:, ['imdb_rating','genre', 'runtime', 'best_pic_nom', 
                  'top200_box', 'director', 'actor1',]]

In [21]:
df_1

,imdb_rating,genre,runtime,best_pic_nom,top200_box,director,actor1
title,,,,,,,
Filly Brown,5.5,Drama,80.0,no,no,Michael D. Olmos,Gina Rodriguez
The Dish,7.3,Drama,101.0,no,no,Rob Sitch,Sam Neill
Waiting for Guffman,7.6,Comedy,84.0,no,no,Christopher Guest,Christopher Guest
The Age of Innocence,7.2,Drama,139.0,no,no,Martin Scorsese,Daniel Day-Lewis
Malevolence,5.1,Horror,90.0,no,no,Stevan Mena,Samantha Dark
Old Partner,7.8,Documentary,78.0,no,no,Chung-ryoul Lee,Choi Won-kyun
Lady Jane,7.2,Drama,142.0,no,no,Trevor Nunn,Cary Elwes
Mad Dog Time,5.5,Drama,93.0,no,no,Larry Bishop,Richard Dreyfuss
Beauty Is Embarrassing,7.5,Documentary,88.0,no,no,Neil Berkeley,Paul Reubens


In [22]:
#Let's also check the column-wise distribution of null values
print(df_1.isnull().values.sum())
print(df_1.isnull().sum())

5
imdb_rating     0
genre           0
runtime         1
best_pic_nom    0
top200_box      0
director        2
actor1          2
dtype: int64


In [23]:
#Dropping missing values from my dataset
df_1.dropna(how='any', inplace=True)
print(df_1.isnull().values.sum()) #checking for missing values after the dropna()

0


In [36]:
#Splitting for 2 matrices: independent variables used for prediction and dependent variables (that is predicted)
X = df_1.drop(["imdb_rating", 'runtime'], axis = 1)   #Feature Matrix
y = df_1["imdb_rating"] #Dependent Variables

In [37]:
X.dtypes

genre           category
best_pic_nom    category
top200_box      category
director          object
actor1            object
dtype: object

In [28]:
X.select_dtypes(['category']).astype('object')
X.dtypes

genre           category
runtime          float64
best_pic_nom    category
top200_box      category
director          object
actor1            object
dtype: object

In [31]:
#Treating categorical variables with One-hot-encoding
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [35]:
class MultiColumnLabelEncoder:
    
    def __init__(self, columns = None):
        self.columns = columns # list of column to encode
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        
        output = X.copy()
        
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname, col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        
        return output
    def fit_transform(self, X, y=None):
        return self.fit(X, y).transform(X)

In [38]:
le = MultiColumnLabelEncoder()

In [39]:
X_train_le = le.fit_transform(X)
X_train_le.head()

,genre,best_pic_nom,top200_box,director,actor1
title,,,,,
Filly Brown,5,0,0,317,157
The Dish,5,0,0,404,397
Waiting for Guffman,3,0,0,88,77
The Age of Innocence,5,0,0,306,95
Malevolence,6,0,0,459,400


In [41]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse=False)
X_train_ohe = ohe.fit_transform(X_train_le)

/Users/marialavrovskaa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [79]:
X_train_ohe
X_train_ohe_col = X_train_ohe[:, np.newaxis]

In [73]:
from sklearn.preprocessing import StandardScaler
#Treating continous variables with Standart Scaler
columns_to_scale = np.array(df_1['runtime'])


array([ 80., 101.,  84., 139.,  90.,  78., 142.,  93.,  88., 119., 127.,
       108., 110., 100., 111.,  96., 140., 106.,  91.,  90., 124.,  82.,
       107., 116.,  87.,  97., 178.,  84., 115., 100.,  85.,  87.,  85.,
       124., 112., 108., 135., 115., 137.,  99., 106., 130.,  98.,  94.,
       154., 113., 115., 135., 106., 125.,  65., 123., 119., 120., 162.,
       140., 115., 102.,  79., 108.,  98., 100., 106.,  92., 115.,  87.,
       103., 127.,  90., 119., 135.,  91., 101.,  93., 101., 130., 101.,
        99., 100., 106., 116.,  90., 117.,  83., 140., 106.,  94.,  91.,
       108., 100., 126., 107.,  94.,  94.,  97., 109., 104., 117.,  93.,
       132.,  95., 102., 101., 105.,  84., 102.,  90., 122., 107.,  97.,
       165., 109., 126.,  89., 102., 123.,  96., 127., 103., 111., 120.,
        86.,  99., 101., 121.,  91.,  92.,  95.,  93.,  98.,  39., 107.,
        91., 157., 118., 126.,  88.,  94.,  92., 100.,  91.,  92., 100.,
        96., 155.,  90., 113., 138., 118., 115., 12

In [76]:
#Initiate Scaler:
scaler = StandardScaler()
scaled_columns  = scaler.fit_transform(columns_to_scale[:, np.newaxis])

In [77]:
scaled_columns

array([[-1.33225187],
       [-0.24884887],
       [-1.12588939],
       [ 1.71159466],
       [-0.81634568],
       [-1.43543311],
       [ 1.86636652],
       [-0.66157382],
       [-0.91952692],
       [ 0.67978228],
       [ 1.09250723],
       [ 0.11228547],
       [ 0.2154667 ],
       [-0.30043949],
       [ 0.26705732],
       [-0.50680196],
       [ 1.76318528],
       [ 0.00910423],
       [-0.76475506],
       [-0.81634568],
       [ 0.93773537],
       [-1.22907063],
       [ 0.06069485],
       [ 0.52501042],
       [-0.97111754],
       [-0.45521135],
       [ 3.7236288 ],
       [-1.12588939],
       [ 0.4734198 ],
       [-0.30043949],
       [-1.07429877],
       [-0.97111754],
       [-1.07429877],
       [ 0.93773537],
       [ 0.31864794],
       [ 0.11228547],
       [ 1.50523218],
       [ 0.4734198 ],
       [ 1.60841342],
       [-0.35203011],
       [ 0.00910423],
       [ 1.24727909],
       [-0.40362073],
       [-0.6099832 ],
       [ 2.48545395],
       [ 0

In [85]:
scaled_columns.shape



(646, 1)

(646, 1, 1028)

In [88]:
X_train_ohe.shape

(646, 1028)

ValueError: all the input array dimensions except for the concatenation axis must match exactly

In [94]:
# Resulted feature matrix withh 
X_2 = np.concatenate((scaled_columns,X_train_ohe),axis=1)

In [135]:
from sklearn import linear_model
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_2, y,test_size = 0.25, random_state = 0)
# Create linear regression object
regr = linear_model.LinearRegression(fit_intercept=False) # Do not use fit_intercept = False if you have removed 1 column after dummy encoding
# Train the model using the training sets
regr.fit(X_train, Y_train)

LinearRegression(copy_X=True, fit_intercept=False, n_jobs=None, normalize=False)

In [136]:
y_pred = regr.predict(X_test)

In [137]:
#Slope for every predictor
df = pd.DataFrame({'Actual': Y_test.values.flatten(), 'Predicted': y_pred.flatten()})
df.head(25)

,Actual,Predicted
0,5.7,6.121768
1,5.9,5.268809
2,6.3,5.774169
3,4.7,5.276988
4,6.2,5.615752
5,5.7,7.432994
6,5.6,6.334758
7,4.8,5.965752
8,4.5,5.276988
9,6.7,5.986173


In [144]:
from sklearn.metrics import mean_squared_error, r2_score
print("Mean squared error: %.2f"
      % mean_squared_error(Y_test, y_pred))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(Y_test, y_pred))

Mean squared error: 1.20
Variance score: -0.07


In [139]:
from sklearn.metrics import explained_variance_score
from sklearn import metrics
explained_variance_score(Y_test, y_pred) 
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(Y_test, y_pred)))

Root Mean Squared Error: 1.0967486054282736


In [128]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

In [130]:
rmse(y_pred, Y_test)

1.0967486054282736

In [140]:
Y_test

title
Bounce                                               5.7
Alligator                                            5.9
North Sea Hijack (ffolkes) (Assault Force)           6.3
A Haunted House 2                                    4.7
A Good Old Fashioned Orgy                            6.2
A Stranger Among Us                                  5.7
Failure to Launch                                    5.6
V.I. Warshawski                                      4.8
My Baby's Daddy                                      4.5
The Long Kiss Goodnight                              6.7
The Science of Sleep                                 7.3
Max Manus: Man of War                                7.3
The Recruit                                          6.6
Oh, God! Book II                                     5.1
Night and the City                                   5.8
Politiki Kouzina (A Touch of Spice)                  7.6
Hard Candy                                           7.1
The Sure Thing           